In [ ]:
import stltovoxel
import os
import h5py

### This example requires STL file for the F1 car model
### You can download your own or 
### Visit https://grabcad.com/library/formula1-2026-fia-showcar-1 and download F1_2026 v40.stl if it still exists
### Credits to user named `Helmar`

folder_path = "./f1car"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created.")
else:
    print(f"Folder '{folder_path}' already exists.")

resolution = 400
stltovoxel.convert_file('F1_2026 v40.stl', 'f1car/f1car.png', resolution=resolution)

Folder './f1car' already exists.
Processing layer 0
Processing layer 1
Processing layer 2
Processing layer 3
Processing layer 4
Processing layer 5
Processing layer 6
Processing layer 7
Processing layer 8
Processing layer 9
Processing layer 10
Processing layer 11
Processing layer 12
Processing layer 13
Processing layer 14
Processing layer 15
Processing layer 16
Processing layer 17
Processing layer 18
Processing layer 19
Processing layer 20
Processing layer 21
Processing layer 22
Processing layer 23
Processing layer 24
Processing layer 25
Processing layer 26
Processing layer 27
Processing layer 28
Processing layer 29
Processing layer 30
Processing layer 31
Processing layer 32
Processing layer 33
Processing layer 34
Processing layer 35
Processing layer 36
Processing layer 37
Processing layer 38
Processing layer 39
Processing layer 40
Processing layer 41
Processing layer 42
Processing layer 43
Processing layer 44
Processing layer 45
Processing layer 46
Processing layer 47
Processing layer 

In [ ]:
import numpy as np
from skimage.measure import block_reduce
from PIL import Image

image_list = []
for i in range(resolution + 2):
    filename = f'f1car/f1car_{i:03d}.png'

    img = Image.open(filename)
    img_array = np.array(img, dtype=np.int32)
    img_array = np.transpose(img_array)
    image_list.append(img_array)

image_stack = np.stack(image_list, axis=0)

# image_stack = np.transpose(image_stack, (0, 0, 2))

ones = np.where(image_stack==1)
zeros = np.where(image_stack==0)
image_stack_inverted = image_stack.copy()
image_stack_inverted[ones] = 0
image_stack_inverted[zeros] = 1

downsample_factor = 0.25
new_shape = tuple(int(dim * downsample_factor) for dim in image_stack_inverted.shape)

downsampled_array = image_stack_inverted

downsampled_array = block_reduce(image_stack_inverted, block_size=(
    image_stack_inverted.shape[0] // new_shape[0],
    image_stack_inverted.shape[1] // new_shape[1],
    image_stack_inverted.shape[2] // new_shape[2]
), func=np.mean)

downsampled_array = np.where(downsampled_array > 0.99, 1, 0)
downsampled_array[:, :, -1] = 1
downsampled_array[-1, :, :] = 1
downsampled_array[:, -1, :] = 1
downsampled_array = np.flip(downsampled_array, -1)

padding = ((20, 25), (50, 50), (50, 250))
downsampled_array = np.pad(downsampled_array, pad_width=padding, mode='constant', constant_values=1)

print(downsampled_array.shape)

with h5py.File("binary_media.h5", "w") as hdf5_file:
    hdf5_file.create_dataset("binary_media", data = downsampled_array.transpose())


(146, 274, 781)
